In [1]:
import os
import torch

import utils

%load_ext autoreload
%autoreload 2

CUDA_LAUNCH_BLOCKING=1
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
print(os.listdir("../data/"))


['kawiki.txt', 'data.txt', 'data2.txt']


In [2]:
from utils import GeorgianLanguageDatasetLoader
import gc
gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = GeorgianLanguageDatasetLoader("../data/kawiki.txt", 10, device)

In [3]:
vocab = dataset.get_vocabulary()
print(vocab.get_itos()[:20])

['<unk>', ',', '.', 'და', ')', '(', '—', '“', '„', ':', 'იყო', 'წელს', 'წლის', 'რომელიც', 'რომ', 'საქართველოს', ';', '-', 'ამ', 'მისი']


In [4]:
train_data, valid_data, test_data = dataset.get_data()
train_data[:10]

tensor([  600,    82, 15948,  7157,  3938,     9,  6244,  4313,  3169,  2557])

In [5]:
' '.join([vocab.get_itos()[i] for i in train_data[40:60]])

'რომელიც კონკურენციას უწევს თავის უფრო ადრეულ ანალოგებს Abobe Flex და Microsoft Silverlight . მოგვიანებით ჩნდება ( ალბათ , ქრისტიანობის და'

In [6]:
text_pipeline = dataset.get_text_pipeline()
text_pipeline("შავი კაცი მიდიოდა")

[263, 395, 0]

In [7]:
train_data_batched, val_data_batched = dataset.get_batched_data()

In [8]:
train_data_batched.shape, train_data_batched.device

(torch.Size([10, 8911]), device(type='cuda', index=0))

In [9]:
x, y = utils.get_batch(train_data_batched, 1)

In [10]:
x.shape, y.shape, train_data_batched.shape

(torch.Size([10, 10]), torch.Size([10, 10]), torch.Size([10, 8911]))

In [11]:
x.storage().data_ptr() == train_data_batched.storage().data_ptr()

True

In [12]:
x[0], y[0]

(tensor([   82, 15948,  7157,  3938,     9,  6244,  4313,  3169,  2557,  1889],
        device='cuda:0'),
 tensor([15948,  7157,  3938,     9,  6244,  4313,  3169,  2557,  1889,     1],
        device='cuda:0'))

In [13]:
%reload_ext autoreload

model = utils.LSTMModel(50, 300, len(vocab), device, 1).cuda()
model = model.to(device)
utils.train_loop(model, train_data_batched, batch_size=10)

Epoch 100 | Iter 100 | Avg Train Loss 2.6842257671058176 | Dev Perplexity None | LR  0.001
Epoch 200 | Iter 200 | Avg Train Loss 0.0628085658699274 | Dev Perplexity None | LR  0.001
Epoch 300 | Iter 300 | Avg Train Loss 0.01895729240961373 | Dev Perplexity None | LR  0.001
Epoch 400 | Iter 400 | Avg Train Loss 0.01177434641867876 | Dev Perplexity None | LR  0.001
Epoch 500 | Iter 500 | Avg Train Loss 0.009123184657655657 | Dev Perplexity 1316171.6820891197 | LR  0.001


In [14]:
text = utils.generate_text(model,device, dataset.vocab, 'იკვლევდა ევროპის და განსაკუთრებით', 20)

In [15]:
print(text)

იკვლევდა ევროპის და განსაკუთრებით ტექნოლოგიის ტექნოლოგიის 16-დან . სლოვაკური უკანანუნისმიერ . შეესაბამება მჟღერ . ( თანხმოვანს . შეესაბამება თანხმოვანს უკანანუნისმიერ , მჟღერ , კბილი


In [16]:
torch.cuda.empty_cache()

In [17]:
geo_model = utils.GeorgianWord2Vec()
geo_model.train("../data/kawiki.txt")

Initializing data


[nltk_data] Downloading package punkt to /home/misho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Model created!


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
embeddings = torch.FloatTensor(geo_model.get_model().wv.vectors)
embeddings.shape

torch.Size([50778, 100])

In [19]:
model = utils.LSTMModel(50, 300, len(vocab), device, 1, embeddings=embeddings).cuda()
model = model.to(device)
utils.train_loop(model, train_data_batched, batch_size=10)

Epoch 100 | Iter 100 | Avg Train Loss 4.331844322681427 | Dev Perplexity None | LR  0.001
Epoch 200 | Iter 200 | Avg Train Loss 2.2769841063022613 | Dev Perplexity None | LR  0.001
Epoch 300 | Iter 300 | Avg Train Loss 1.2310896229743957 | Dev Perplexity None | LR  0.001
Epoch 400 | Iter 400 | Avg Train Loss 0.6331655657291413 | Dev Perplexity None | LR  0.001
Epoch 500 | Iter 500 | Avg Train Loss 0.37198588371276853 | Dev Perplexity 149205150123.07108 | LR  0.001


In [20]:
text = utils.generate_text(model,device, dataset.vocab, 'სიცოცხლის ბოლო წლები', 20)
print(text)

სიცოცხლის ბოლო წლები , ახალი ეჭვის , , სწავლობდა . . სწავლობდა აღმოჩენილიქნა აღმოჩენილიქნა . სწავლობდა აღმოჩენილიქნა სწავლობდა , სწავლობდა . , ,
